In [25]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import Ridge
import datetime
import requests
import json
import joblib
from tvDatafeed import TvDatafeed, Interval

In [14]:
def getdata(name):
    data=tv.get_hist(symbol=name,exchange='BINANCE',interval=Interval.in_daily,n_bars=10)
    data=data.reset_index()
    data["datetime"]=data["datetime"].dt.strftime('%Y-%m-%d')
    data["datetime"] = pd.to_datetime(data["datetime"])
    return data

In [15]:
def getapi(api):
    response_API=requests.get(api)
    data=response_API.text
    parse_json=json.loads(data)
    df = pd.json_normalize(parse_json['values'])
    listtime=list(df['x'])
    listtime2=[]
    for i in listtime:
        listtime2.append(datetime.datetime.fromtimestamp(i).strftime('%Y-%m-%d'))
    df['datetime']=pd.Series(listtime2)
    df["datetime"] = pd.to_datetime(df["datetime"])
    df=df.drop(columns=['x'])
    df.rename(columns={'y':'value'},inplace=True)
    df=df.drop_duplicates(subset=['datetime'])
    return df

In [40]:
tv = TvDatafeed()
bitcoin=getdata('BTCUSDT')
eth=getdata('ETHUSDT')
ada=getdata('ADAUSDT')

In [41]:
number_transactions=getapi('https://api.blockchain.info/charts/n-transactions?timespan=10days&rollingAverage=24hours&format=json&sampled=false')
number_address=getapi('https://api.blockchain.info/charts/n-unique-addresses?timespan=10days&rollingAverage=24hours&format=json&sampled=false')
transaction_second=getapi('https://api.blockchain.info/charts/transactions-per-second?timespan=10days&rollingAverage=24hours&format=json&sampled=false')
transaction_second=transaction_second.groupby(['datetime']).mean().reset_index()
total_bitcoin=getapi('https://api.blockchain.info/charts/total-bitcoins?timespan=10days&rollingAverage=24hours&format=json&sampled=true')
hash_rate=getapi('https://api.blockchain.info/charts/hash-rate?timespan=10days&rollingAverage=24hours&format=json&sampled=false')

In [42]:
bitcoin=bitcoin[["datetime","open","close"]]
bitcoin.tail(5)

,datetime,open,close
5,2022-04-01,46283.49,45811.00
6,2022-04-02,45810.99,46407.35
7,2022-04-03,46407.36,46580.51
8,2022-04-04,46580.50,45497.55
9,2022-04-05,45497.54,45285.03


In [43]:
bitcoin=bitcoin.merge(eth[["datetime","open"]],on='datetime',how='left',suffixes=('_bitcoin', '_eth'))
bitcoin=bitcoin.merge(ada[["datetime","open"]],on='datetime',how='left')
# bitcoin=bitcoin.merge(ada[["datetime","open"]],on='datetime',how='left',suffixes=('_bnb', '_ada'))
bitcoin

,datetime,open_bitcoin,close,open_eth,open
0,2022-03-27,46827.76,47122.21,3295.65,1.184
1,2022-03-28,47122.21,47434.80,3332.91,1.170
2,2022-03-29,47434.79,47067.99,3400.51,1.191
3,2022-03-30,47067.99,45510.34,3385.80,1.189
4,2022-03-31,45510.35,46283.49,3281.51,1.141
5,2022-04-01,46283.49,45811.00,3455.20,1.165
6,2022-04-02,45810.99,46407.35,3443.77,1.156
7,2022-04-03,46407.36,46580.51,3521.90,1.186
8,2022-04-04,46580.50,45497.55,3519.50,1.212
9,2022-04-05,45497.54,45285.03,3407.00,1.170


In [44]:
data_api=number_transactions.merge(number_address,on='datetime',how='left',suffixes=('_number_transaction', '_number_address'))
data_api=data_api.merge(transaction_second,on='datetime',how='left')
data_api=data_api.merge(total_bitcoin,on='datetime',how='left',suffixes=('_transaction_second', '_total_bitcoin'))
data_api=data_api.merge(hash_rate,on='datetime',how='left')
data_api.rename(columns={'value':'value_hash_rate'},inplace=True)

In [45]:
bitcoin=bitcoin.merge(data_api,on='datetime',how='left')
bitcoin.rename(columns={'open':'open_ada'},inplace=True)
bitcoin.rename(columns={'close':'close_bitcoin'},inplace=True)
bitcoin

,datetime,open_bitcoin,close_bitcoin,open_eth,open_ada,value_number_transaction,value_number_address,value_transaction_second,value_total_bitcoin,value_hash_rate
0,2022-03-27,46827.76,47122.21,3295.65,1.184,293304.0,720718.0,2.643056,1.899458e+07,2.169844e+08
1,2022-03-28,47122.21,47434.80,3332.91,1.170,286940.0,706611.0,2.576562,1.899536e+07,2.060669e+08
2,2022-03-29,47434.79,47067.99,3400.51,1.191,272864.0,662686.0,3.422396,1.899640e+07,1.842320e+08
3,2022-03-30,47067.99,45510.34,3385.80,1.189,279210.0,711182.0,3.371007,1.899737e+07,1.942474e+08
4,2022-03-31,45510.35,46283.49,3281.51,1.141,284105.0,738464.0,3.218229,1.899823e+07,2.003716e+08
5,2022-04-01,46283.49,45811.00,3455.20,1.165,254442.0,660755.0,3.338715,1.899911e+07,1.918452e+08
6,2022-04-02,45810.99,46407.35,3443.77,1.156,225564.0,585946.0,3.327604,1.899993e+07,1.975295e+08
7,2022-04-03,46407.36,46580.51,3521.90,1.186,275106.0,691360.0,2.791319,1.900079e+07,2.245299e+08
8,2022-04-04,46580.50,45497.55,3519.50,1.212,273430.0,692888.0,2.592361,1.900166e+07,2.060559e+08
9,2022-04-05,45497.54,45285.03,3407.00,1.170,NaN,NaN,NaN,NaN,NaN


In [46]:
bitcoin.isnull().sum()

datetime                    0
open_bitcoin                0
close_bitcoin               0
open_eth                    0
open_ada                    0
value_number_transaction    1
value_number_address        1
value_transaction_second    1
value_total_bitcoin         1
value_hash_rate             1
dtype: int64

In [47]:
temp=bitcoin["datetime"]
bitcoin=bitcoin.drop(columns=['datetime'])
bitcoin=bitcoin.interpolate()
bitcoin['datetime']=temp
bitcoin.isnull().sum()

open_bitcoin                0
close_bitcoin               0
open_eth                    0
open_ada                    0
value_number_transaction    0
value_number_address        0
value_transaction_second    0
value_total_bitcoin         0
value_hash_rate             0
datetime                    0
dtype: int64

In [48]:
bitcoin

,open_bitcoin,close_bitcoin,open_eth,open_ada,value_number_transaction,value_number_address,value_transaction_second,value_total_bitcoin,value_hash_rate,datetime
0,46827.76,47122.21,3295.65,1.184,293304.0,720718.0,2.643056,1.899458e+07,2.169844e+08,2022-03-27
1,47122.21,47434.80,3332.91,1.170,286940.0,706611.0,2.576562,1.899536e+07,2.060669e+08,2022-03-28
2,47434.79,47067.99,3400.51,1.191,272864.0,662686.0,3.422396,1.899640e+07,1.842320e+08,2022-03-29
3,47067.99,45510.34,3385.80,1.189,279210.0,711182.0,3.371007,1.899737e+07,1.942474e+08,2022-03-30
4,45510.35,46283.49,3281.51,1.141,284105.0,738464.0,3.218229,1.899823e+07,2.003716e+08,2022-03-31
5,46283.49,45811.00,3455.20,1.165,254442.0,660755.0,3.338715,1.899911e+07,1.918452e+08,2022-04-01
6,45810.99,46407.35,3443.77,1.156,225564.0,585946.0,3.327604,1.899993e+07,1.975295e+08,2022-04-02
7,46407.36,46580.51,3521.90,1.186,275106.0,691360.0,2.791319,1.900079e+07,2.245299e+08,2022-04-03
8,46580.50,45497.55,3519.50,1.212,273430.0,692888.0,2.592361,1.900166e+07,2.060559e+08,2022-04-04
9,45497.54,45285.03,3407.00,1.170,273430.0,692888.0,2.592361,1.900166e+07,2.060559e+08,2022-04-05


In [49]:
selected_features=['open_bitcoin', 'open_eth', 'open_ada', 'value_number_transaction',
       'value_number_address', 'value_transaction_second',
       'value_total_bitcoin', 'value_hash_rate', 'weighted_sentiment']
model_from_joblib = joblib.load('savedmodel.pkl')
prediction = model_from_joblib.predict(bitcoin[selected_features])

KeyError: "['weighted_sentiment'] not in index"